##### Dependencies and Dataset

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# for modelling
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [62]:
# import dataset
df = pd.read_csv('StudentsPerformance.csv')

In [63]:
df.head(2)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88


In [64]:
df.columns = [col.replace(" ", "_") for col in df.columns ]
df.columns = [col.replace("/", "_") for col in df.columns ]
df.head(3)

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93


##### Prepare X and y variables

In [65]:
X = df.drop(columns = ['math_score'], axis = 1)
X.head(2)

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88


In [66]:
print("Categories for 'gender' variable: ", end = " ")
print(df.gender.unique())
print()

print("Categories for 'lunch' variable: ", end = " ")
print(df.lunch.unique())
print()

print("Categories for 'parental_level_of_education' variable: ", end = " ")
print(df.parental_level_of_education.unique())
print()

print("Categories for 'race_ethnicity' variable: ", end = " ")
print(df.race_ethnicity.unique())
print()

print("Categories for 'test_preparation_course' variable: ", end = " ")
print(df.test_preparation_course.unique())
print()

Categories for 'gender' variable:  ['female' 'male']

Categories for 'lunch' variable:  ['standard' 'free/reduced']

Categories for 'parental_level_of_education' variable:  ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']

Categories for 'race_ethnicity' variable:  ['group B' 'group C' 'group A' 'group D' 'group E']

Categories for 'test_preparation_course' variable:  ['none' 'completed']



In [67]:
y = df.math_score
y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math_score, Length: 1000, dtype: int64

In [68]:
# Create Column Transformer with 3 types of transformers
num_features = X.select_dtypes(exclude='object').columns
cat_features = X.select_dtypes(include='object').columns

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
        ("StandardScaler", numeric_transformer, num_features)
    ]
)

In [69]:
X = preprocessor.fit_transform(X)

In [70]:
X.shape

(1000, 19)

In [71]:
# separate dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((800, 19), (200, 19))

#### Create an Evaluation function to give all metrics after model training

In [76]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [77]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge":Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(),
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Evaluate trainin and test dataset
    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)
    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model performance for training set')
    print("- Root Mean Square Error: {:.4f}".format(model_train_rmse))
    print("- Mean Asolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('------------------------------')

    print('Model performance for test set')
    print("- Root Mean Square Error: {:.4f}".format(model_test_rmse))
    print("- Mean Asolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)

    print('='*35)
    print('\n')

Linear Regression
Model performance for training set
- Root Mean Square Error: 5.3257
- Mean Asolute Error: 4.2698
- R2 Score: 0.8742
------------------------------
Model performance for test set
- Root Mean Square Error: 5.4212
- Mean Asolute Error: 4.2345
- R2 Score: 0.8792


Lasso
Model performance for training set
- Root Mean Square Error: 6.5938
- Mean Asolute Error: 5.2063
- R2 Score: 0.8071
------------------------------
Model performance for test set
- Root Mean Square Error: 6.5197
- Mean Asolute Error: 5.1579
- R2 Score: 0.8253


Ridge
Model performance for training set
- Root Mean Square Error: 5.3233
- Mean Asolute Error: 4.2650
- R2 Score: 0.8743
------------------------------
Model performance for test set
- Root Mean Square Error: 5.3904
- Mean Asolute Error: 4.2111
- R2 Score: 0.8806


K-Neighbors Regressor
Model performance for training set
- Root Mean Square Error: 5.7079
- Mean Asolute Error: 4.5168
- R2 Score: 0.8555
------------------------------
Model performance 

#### Results

In [80]:
pd.DataFrame(list(zip(model_list, r2_list)),
            columns = ['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],
            ascending = False)

,Model Name,R2_Score
2,Ridge,0.880593
0,Linear Regression,0.879222
7,CatBoosting Regressor,0.851632
8,AdaBoost Regressor,0.850898
5,Random Forest Regressor,0.850762
6,XGBRegressor,0.827797
1,Lasso,0.825320
3,K-Neighbors Regressor,0.783813
4,Decision Tree,0.740957


# 